# Smoothing Moving Averages

Re-computing moving averages on live data is error-prone, due to close-bias (explain)


In [127]:
# Parameters
symbol = 'BTCUSDT'
start_date = '2021-01-01'
interval = '15m'
sample_size = 1000
binance_request_limit=1000

## Setup

In [128]:
# Imports
import requests
import numpy as np
import pandas as pd

In [129]:
# Helper functions

def fetch_klines(interval, start_timestamp, limit, close_timestamp = None):
  if (limit > binance_request_limit):
    raise("Given limit greater than Binance's page size")

  url = f'https://api.binance.com/api/v3/klines?symbol={symbol}&interval={interval}&limit={limit}&startTime={start_timestamp}'

  if (close_timestamp != None):
    url += f'&endTime={close_timestamp}'

  response = requests.get(url)
  return response.json()

def make_kline_df(kline_data):
  kline_df = pd.DataFrame(map(lambda kline: kline[:7], kline_data), columns=['open_timestamp','open','high','low','close','volume','close_timestamp'])

  kline_df['open_time'] = pd.to_datetime(kline_df['open_timestamp'], unit='ms')
  kline_df['close_time'] = pd.to_datetime(kline_df['close_timestamp'], unit='ms')

  interval = kline_df.head(1).apply(lambda row: row['close_timestamp'] - row['open_timestamp'] + 1, axis = 1)[0]
  kline_df['interval'] = interval

  kline_df = kline_df.set_index(['open_timestamp', 'close_timestamp'])
  
  return kline_df

def to_binance_timestamp(dt):
  return int(dt.timestamp() * 1000)

def chunk(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [130]:
# Fetch working klines

kline_data = fetch_klines(interval, to_binance_timestamp(pd.to_datetime(start_date)), sample_size)
kline_df = make_kline_df(kline_data)
kline_df


# base_kline_data = []
# while base_kline_data.count == 0 or base_kline_data[-1:][0]:

# base_kline_response = fetch_klines('1m', start_timestamp)
# base_kline_data = base_kline_response.json()

,,open,high,low,close,volume,open_time,close_time,interval
open_timestamp,close_timestamp,,,,,,,,
1609459200000,1609460099999,28923.63000000,29017.50000000,28690.17000000,28752.80000000,840.07756900,2021-01-01 00:00:00,2021-01-01 00:14:59.999,900000
1609460100000,1609460999999,28752.80000000,28875.55000000,28720.91000000,28836.63000000,480.61117900,2021-01-01 00:15:00,2021-01-01 00:29:59.999,900000
1609461000000,1609461899999,28836.63000000,28943.87000000,28836.62000000,28930.11000000,471.13470800,2021-01-01 00:30:00,2021-01-01 00:44:59.999,900000
1609461900000,1609462799999,28930.11000000,29031.34000000,28889.99000000,28995.13000000,519.98798900,2021-01-01 00:45:00,2021-01-01 00:59:59.999,900000
1609462800000,1609463699999,28995.13000000,29385.00000000,28960.35000000,29382.59000000,1735.83900500,2021-01-01 01:00:00,2021-01-01 01:14:59.999,900000
...,...,...,...,...,...,...,...,...,...
1610354700000,1610355599999,35766.96000000,35919.86000000,35359.01000000,35430.00000000,1908.99692600,2021-01-11 08:45:00,2021-01-11 08:59:59.999,900000
1610355600000,1610356499999,35430.00000000,35570.00000000,34800.00000000,35136.99000000,2232.89687800,2021-01-11 09:00:00,2021-01-11 09:14:59.999,900000
1610356500000,1610357399999,35135.24000000,35579.74000000,35017.83000000,35546.90000000,1229.92993100,2021-01-11 09:15:00,2021-01-11 09:29:59.999,900000


In [131]:
# Fetch 1m klines, for analysis of the inner increments of a kline

incremental_kline_interval = '1m'
incremental_kline_interval_ms = 60 * 1000

open_timestamp = kline_df.head(1).index.values[0][0]
close_timestamp = kline_df.tail(1).index.values[0][1]

open_timestamps = list(range(open_timestamp, close_timestamp, incremental_kline_interval_ms))
open_timestamp_batches = list(chunk(open_timestamps, binance_request_limit))

incremental_kline_data = list()
for open_timestamp_batch in open_timestamp_batches:
  curr_open_timestamp = open_timestamp_batch[0]
  curr_close_timestamp = open_timestamp_batch[-1:][0]
  curr_incremental_kline_data = fetch_klines(incremental_kline_interval, curr_open_timestamp, binance_request_limit, curr_close_timestamp)
  incremental_kline_data = incremental_kline_data + list(curr_incremental_kline_data)

incremental_kline_df = make_kline_df(incremental_kline_data)
incremental_kline_df

,,open,high,low,close,volume,open_time,close_time,interval
open_timestamp,close_timestamp,,,,,,,,
1609459200000,1609459259999,28923.63000000,28961.66000000,28913.12000000,28961.66000000,27.45703200,2021-01-01 00:00:00,2021-01-01 00:00:59.999,60000
1609459260000,1609459319999,28961.67000000,29017.50000000,28961.01000000,29009.91000000,58.47750100,2021-01-01 00:01:00,2021-01-01 00:01:59.999,60000
1609459320000,1609459379999,29009.54000000,29016.71000000,28973.58000000,28989.30000000,42.47032900,2021-01-01 00:02:00,2021-01-01 00:02:59.999,60000
1609459380000,1609459439999,28989.68000000,28999.85000000,28972.33000000,28982.69000000,30.36067700,2021-01-01 00:03:00,2021-01-01 00:03:59.999,60000
1609459440000,1609459499999,28982.67000000,28995.93000000,28971.80000000,28975.65000000,24.12433900,2021-01-01 00:04:00,2021-01-01 00:04:59.999,60000
...,...,...,...,...,...,...,...,...,...
1610358900000,1610358959999,35519.98000000,35530.08000000,35480.18000000,35502.64000000,66.63764400,2021-01-11 09:55:00,2021-01-11 09:55:59.999,60000
1610358960000,1610359019999,35502.63000000,35600.00000000,35489.00000000,35591.68000000,96.73938600,2021-01-11 09:56:00,2021-01-11 09:56:59.999,60000
1610359020000,1610359079999,35591.67000000,35666.00000000,35587.28000000,35641.47000000,93.97607400,2021-01-11 09:57:00,2021-01-11 09:57:59.999,60000
